In [31]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the model
model = load_model("15_point_plastic_classifier_model8.keras")
# model = load_model("15_point_plastic_classifier_modelVPP_PET.keras")
# model = load_model("3_point_plastic_classifier_model.keras")



In [32]:
# # Sample data based on your provided example (modify this as needed)
# sample_data = [
#     [2493065, 4558145, 7638499, 8387631, 8388397, 8389475, 4054559, 2106185, 3340801, 5935779, 8389399, 8388961, 8386511, 6563979, 3097997]
# ]

# # Create a function to generate a representative dataset
# def representative_dataset():
#     for data_point in sample_data:
#         # Reshape and convert to float32 (modify this if your model's input format is different)
#         input_data = tf.constant(data_point, shape=(1, 15), dtype=tf.float32)
#         yield [input_data]

# # Convert the model to TFLite format with quantization
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
# converter.representative_dataset = representative_dataset  # Specify the representative dataset

# tflite_model = converter.convert()

# # Save the TensorFlow Lite model
# with open('your_model.tflite', 'wb') as f:
#     f.write(tflite_model)

In [33]:
# data = pd.read_csv("AllTogetherPE.csv")
data = pd.read_csv("resampled_dataset8_PVC.csv")
# data = pd.read_csv("resampled_datasetVPP_PET.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets (Normal)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=54)


# Normalize the manual input if needed
X_test_normalized = ((X_test - 0) / (8410000 - 0))
# X_test_normalized = (X_test - X_train.min()) / (X_train.max() - X_train.min())

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -5:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(5, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][1-j]:.2f}% likelihood")



200/200 [==============================] - 0s 523us/step
Test Accuracy: 0.9740625023841858
Sample 1: Correct, it was PET
  Prediction 1: PET with 0.00% likelihood
  Prediction 2: Unknown with 0.00% likelihood
  Prediction 3: PE with 100.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 2: Correct, it was PET
  Prediction 1: PET with 0.00% likelihood
  Prediction 2: Unknown with 0.00% likelihood
  Prediction 3: PP with 99.99% likelihood
  Prediction 4: PE with 0.01% likelihood
Sample 3: Correct, it was Unknown
  Prediction 1: Unknown with 0.00% likelihood
  Prediction 2: PET with 0.00% likelihood
  Prediction 3: PE with 100.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 4: Correct, it was PET
  Prediction 1: PET with 0.00% likelihood
  Prediction 2: Unknown with 0.00% likelihood
  Prediction 3: PE with 100.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 5: Correct, it was PET
  Prediction 1: PET with 0.00% likelihood
  Prediction 2: PE with 0.

In [34]:

exclude_alu_indices = y != 'ALU'
X_filtered = X[exclude_alu_indices]
y_filtered = y_encoded[exclude_alu_indices]
# Split the filtered data into train and test sets (Filtered ALU)
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=50)
#features
X_test_normalized = ((X_test - 0) / (8410000 - 0))

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -4:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(4, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][3-j]:.2f}% likelihood")


200/200 [==============================] - 0s 528us/step
Test Accuracy: 0.9775000214576721
Sample 1: Correct, it was Unknown
  Prediction 1: Unknown with 100.00% likelihood
  Prediction 2: PP with 0.00% likelihood
  Prediction 3: PET with 0.00% likelihood
  Prediction 4: PE with 0.00% likelihood
Sample 2: Correct, it was PET
  Prediction 1: PET with 100.00% likelihood
  Prediction 2: PE with 0.00% likelihood
  Prediction 3: Unknown with 0.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 3: Correct, it was PET
  Prediction 1: PET with 100.00% likelihood
  Prediction 2: PP with 0.00% likelihood
  Prediction 3: Unknown with 0.00% likelihood
  Prediction 4: PE with 0.00% likelihood
Sample 4: Correct, it was PET
  Prediction 1: PET with 100.00% likelihood
  Prediction 2: Unknown with 0.00% likelihood
  Prediction 3: PE with 0.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 5: Correct, it was PET
  Prediction 1: PET with 100.00% likelihood
  Prediction 2: Unknown

In [35]:
# Get manual input from the user
manual_input = np.array([[1271389,2107581,3534859,5988057,6233631,3880377,1498541,985865,1561809,2707983,4719147,6111335,5063453,2701889,1262325]])  # Replace with your input values

# Normalize the manual input if needed
normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

# Make predictions on manual input
predictions = model.predict(normalized_manual_input)
print(predictions)

# Get the top prediction
top_prediction = np.argmax(predictions)
print(top_prediction)

top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
print(top_plastic_type)
likelihood = predictions[0][top_prediction] * 100

# Display the prediction for manual input
print(f"Manual Input Prediction:")
print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

1/1 [==============================] - 0s 42ms/step
[[2.1984098e-02 3.0081870e-35 9.7801596e-01 2.2142071e-22]]
2
PP
Manual Input Prediction:
  Prediction: PP with 97.80% likelihood


In [36]:
import numpy as np  # Add this line to import the NumPy library
import serial
from sklearn.preprocessing import LabelEncoder


# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Initialize a variable to store the received data
received_data = ""

print(label_encoder.classes_)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()
        
        # Append the received line to the variable
        received_data += line

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in received_data.split(',')]

        # Convert the list of values into a NumPy array
        manual_input = np.array([data_values])

        # Normalize the manual input if needed
        normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

        # Make predictions on manual input
        predictions = model.predict(normalized_manual_input)
        # print(predictions)

        # Get the top prediction
        top_prediction = np.argmax(predictions)
        print(top_prediction)

        top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
        print(top_plastic_type)
        likelihood = predictions[0][top_prediction] * 100

        # Display the prediction for manual input
        print(f"Manual Input Prediction:")
        print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

        received_data = ""

        # Send the top plastic type back over serial
        ser.write(top_plastic_type.encode('utf-8'))

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the serial port when done
    ser.close()

# Now, you can use the 'received_data' variable for further processing
print("total data:\n", received_data)


['PE' 'PET' 'PP' 'Unknown']
1/1 [==============================] - 0s 16ms/step
3
Unknown
Manual Input Prediction:
  Prediction: Unknown with 100.00% likelihood
1/1 [==============================] - 0s 15ms/step
2
PP
Manual Input Prediction:
  Prediction: PP with 74.19% likelihood
1/1 [==============================] - 0s 15ms/step
2
PP
Manual Input Prediction:
  Prediction: PP with 97.97% likelihood
1/1 [==============================] - 0s 16ms/step
2
PP
Manual Input Prediction:
  Prediction: PP with 97.84% likelihood
1/1 [==============================] - 0s 15ms/step
3
Unknown
Manual Input Prediction:
  Prediction: Unknown with 67.98% likelihood
1/1 [==============================] - 0s 16ms/step
2
PP
Manual Input Prediction:
  Prediction: PP with 57.50% likelihood
1/1 [==============================] - 0s 15ms/step
3
Unknown
Manual Input Prediction:
  Prediction: Unknown with 61.58% likelihood
1/1 [==============================] - 0s 16ms/step
3
Unknown
Manual Input Prediction:


SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))

In [37]:
import numpy as np
import serial
import csv  # Import the csv module
from sklearn.preprocessing import LabelEncoder

# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Create and open a CSV file for writing
csv_file = open('adding.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        print(data_values)

        # Write the data values to the CSV file as a new row
        csv_writer.writerow(data_values)

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the CSV file and the serial port when done
    csv_file.close()
    ser.close()

# The received data has been saved in the 'received_data.csv' file
print("Data saved to 'PP2.csv'")



[936837.0, 1740459.0, 3205307.0, 7359151.0, 6240375.0, 3779665.0, 1611031.0, 610273.0, 1314025.0, 2478041.0, 5375249.0, 7031893.0, 5207617.0, 2765375.0, 1135267.0]
[1117009.0, 2080693.0, 3510233.0, 7226979.0, 6193253.0, 4192789.0, 1913429.0, 894555.0, 1533009.0, 2741927.0, 5322151.0, 6672287.0, 5179479.0, 3071771.0, 1421053.0]
[1253043.0, 2258847.0, 3611891.0, 6913277.0, 6433173.0, 4544139.0, 2083497.0, 1075339.0, 1648161.0, 2776763.0, 4970735.0, 6294655.0, 5249749.0, 3272015.0, 1589813.0]
[1181477.0, 2028573.0, 3330625.0, 6421237.0, 5851445.0, 4021281.0, 1817865.0, 916113.0, 1455539.0, 2531943.0, 4809381.0, 6280649.0, 5089565.0, 2951741.0, 1386419.0]
[1232859.0, 2199213.0, 3825265.0, 6938309.0, 4590289.0, 2986155.0, 1949939.0, 1137095.0, 1645303.0, 2975531.0, 5448263.0, 5972777.0, 3972593.0, 2534853.0, 1564899.0]
[1122331.0, 2116755.0, 3763279.0, 7750463.0, 7005771.0, 4618625.0, 2004561.0, 914131.0, 1582951.0, 2897027.0, 5711533.0, 7378445.0, 5846847.0, 3329201.0, 1444251.0]
[1180579.

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))